In [6]:
import autograd.numpy as anp
import pymanopt
from pymanopt.manifolds import Euclidean, SpecialOrthogonalGroup
import pymanopt.optimizers


# Set the dimensions of the manifolds
dim = 3

# Creating a eucledian and special orthogonal manifold
manifoldTranslation = Euclidean(dim)
manifoldRotation = SpecialOrthogonalGroup(dim)

# Creating the product manifold on which we perform our optimization (SO(3)xR(3))
productmanifold = pymanopt.manifolds.Product((manifoldRotation, manifoldTranslation))

# Defining the landmarks in the world coordinate frame
Worldlandmarks = anp.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0.,0.,0.]])
# Worldlandmarks = Worldlandmarks.astype(float)

#Landmark points in the image of camera (these points lie in the S(2) manifold)
CameraPoints = anp.array([[0.866, 0.289, -0.408], [-0.945, 0.189, -0.267], [-0.189, -0.567, 0.802], [0.289, -0.866, -0.408]])


@pymanopt.function.autograd(productmanifold)
# defining the cost function as a non-linear least squares
def cost(R, t):
    Distance = 0
    for i in range(4):
        # Landmarks in camera body centric frame using the affine transformation
        TransformedWorldlandmarks = R@Worldlandmarks[i] + t

        # Projecting these points onto the camera image using the projection map
        TransformedWorldlandmarks = TransformedWorldlandmarks / (TransformedWorldlandmarks[0]**2 + TransformedWorldlandmarks[1]**2 
                                    + TransformedWorldlandmarks[2]**2)**0.5
        
        TransformedWorldlandmarks = anp.array(TransformedWorldlandmarks)
        
        # TranslatedWorldlandmarks = TranslatedWorldlandmarks
        
        Distance += anp.arccos(TransformedWorldlandmarks.T@CameraPoints[i])**2
    return Distance

# Defining the optimization problem on the product manifold using the non-linear least squares cost
problem = pymanopt.Problem(productmanifold, cost)

# optimizer to optimize the pose of the camera, by performing gradient descent on the product manifold
optimizer = pymanopt.optimizers.SteepestDescent()
result = optimizer.run(problem)

# print translation and rotational components of the optimized estimated pose of camera
print("Rotation:",result.point[0])
print("Translation:",result.point[1])

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         +1.9042289127910969e+01    5.57366479e+00    
   2         +1.5305522351637485e+01    2.99430247e+00    
   3         +1.1999814734398896e+01    1.17933446e+00    
   4         +8.2636165798856940e+00    3.02533972e-01    
   5         +7.8683469269378818e+00    1.66294152e-01    
   6         +7.3976265216634793e+00    1.07395516e-01    
   7         +7.1689391845064812e+00    1.12201390e-01    
   8         +7.1343297569887616e+00    1.02586936e-01    
   9         +7.0822014352924532e+00    3.68333768e-02    
  10         +7.0790004412219849e+00    5.07417161e-02    
  11         +7.0706023483922351e+00    2.30916742e-02    
  12         +7.0643932582641691e+00    4.42839047e-02    
  13         +7.0559241892332896e+00    3.31969975e-02    
  14         +7.0519829754115140e+00    4.55074763e-02    
  15         +7.0448788400211049e+00    2.